# Time Series in Python
# Practical approach

## План
1. Введение (~15 мин)
  * Основные определения, варианты постановки задачи
  * Немного про кейсы, с какими временными рядами будем работать
  * Метрики качества прогнозов, R^2, MSE, MAE, MAPE, RLMSE и т.д.
2. Движемся, сглаживаем и оцениваем (~40 мин)
  * Rolling window estimations + практика, прогнозируем оконными функциями, считаем качество
  * Экспоненциальное сглаживание, простое, двойное и Holt-Winters + примеры с ручной настройкой параметров
  * Кросс-валидация на временных рядах, подбор параметров + практика, оценка Holt-Winters с минимизацией ошибки на кросс-валидации на разных рядах
3. Эконометрический подход (~40 мин)
  * Стационарность и единичные корни - интуиция теории + практика проверки разных рядов на стационарность ADF, KPSS
  * Избавление от нестационарности, логарифмирование, дифференцирование, сезонное дифференцирование, box-cox, выделение трендов + практика, приводим разные ряды к стационарному виду
  * ARIMA, немного интуиции, выбор начальных параметров модели по ACF, PACF, выбор параметров перебором + парктика на простых рядах (чтобы быстрее считались) и пример SARIMAX с подобранными параметрами
  * Ограничения, недостатки эконометрического подхода (индивидуальная работа с каждым рядом, долгий подбор параметров)
4. Machine Learning для временных рядов (~50 мин)
  * Feature extraction, engineering, предобработка данных, какие признаки и как можно извлечь + практика, формируем датасет для моделей по ряду, пишем функцию для автогенерации признаков
  * Линейные модели, обучаем, смотрим качество на кросс-валидации, оцениваем важность признаков
  * Регуляризация, отбор признаков, сокращение признакового пространства, ridge, lasso, PCA
  * Бустиг для временных рядов - ограничения и преимущества, композиция моделей (xgboost + linear)
5. To Infinity and Beyond (~30 мин)
  * Facebook Prophet - автоматические прогнозы
  * Что еще можно делать с временными рядами - RNN, LSTM 
  * Немного про практические кейсы с работы, какие задачи с временными рядами решал

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# 1) Введение

Небольшое [определение](https://ru.wikipedia.org/wiki/Временной_ряд) временного ряда:
> Временной ряд – это последовательность значений, описывающих протекающий во времени процесс, измеренных в последовательные моменты времени, обычно через равные промежутки

Таким образом, данные оказываются упорядочены относительно неслучайных моментов времени, и, значит, в отличие от случайных выборок, могут содержать в себе дополнительную информацию, которую мы постараемся извлечь. 

In [70]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go
init_notebook_mode(connected = True)

def plotTimeseries(df, title = ''):
    """
        Plot timeseries data in plotly
    """
    data = []
    for column in df.columns:
        trace = go.Scatter(
            x = df.index,
            y = df[column],
            mode = 'lines',
            name = column
        )
        data.append(trace)
    
    layout = dict(title = title)
    fig = dict(data = data, layout = layout)
    iplot(fig, show_link=False)

C какими временными рядами будем работать


In [76]:
ads = pd.read_csv('data/ads.csv', index_col=['Time'], parse_dates=['Time'])
currency = pd.read_csv('data/currency.csv', index_col=['Time'], parse_dates=['Time'])
plotTimeseries(ads, title = "Ads")
plotTimeseries(currency, title = "Currency")

## Метрики качества прогноза

- R квадрат, коэффициент детерминации, процент объясненной моделью дисперсии, $(-\infty, 1]$

$R^2 = 1 - \frac{SS_{res}}{SS_{tot}}$ 

```python
sklearn.metrics.r2_score
```

- Mean Absolute Error, интерпретируемая метрика, измеряется в тех же единицах, что и исходный ряд

$MAE = \frac{\sum\limits_{i=1}^{n} |y_i - \hat{y}_i|}{n}$ 

```python
sklearn.metrics.mean_absolute_error
```

...To be continued

# 2) Движемся, сглаживаем и оцениваем

## Rolling window estimations

Начнем моделирование с наивного предположения - "завтра будет, как вчера", но вместо модели вида $\hat{y}_{t} = y_{t-1}$ будем считать, что будущее значение переменной зависит от среднего $n$ её предыдущих значений, а значит, воспользуемся скользящей средней. 

$\hat{y}_{t} = \frac{1}{k} \displaystyle\sum^{k-1}_{n=0} y_{t-n}$

In [29]:
def moving_average(series, n):
    """
        Calculate average of last n observations
    """
    return np.average(series[-n:])

In [61]:
moving_average(ads, 7)

133498.97959183675

In [83]:
data = ads.copy()
actual = data.values
forecast = data.rolling(window=7).mean().values

In [ ]:
def plotMovingAverage(series, n):

    """
        series - dataframe with timeseries
        n - rolling window size 

    """

    rolling_mean = series.rolling(window=n).mean()

    # При желании, можно строить и доверительные интервалы для сглаженных значений
    #rolling_std =  series.rolling(window=n).std()
    #upper_bond = rolling_mean+1.96*rolling_std
    #lower_bond = rolling_mean-1.96*rolling_std

    plt.figure(figsize=(15,5))
    plt.title("Moving average\n window size = {}".format(n))
    plt.plot(rolling_mean, "g", label="Rolling mean trend")

    #plt.plot(upper_bond, "r--", label="Upper Bond / Lower Bond")
    #plt.plot(lower_bond, "r--")
    plt.plot(dataset[n:], label="Actual values")
    plt.legend(loc="upper left")
    plt.grid(True)

In [27]:
moving_average(ads, n=7)

123608.57142857143

# 3) Эконометрический подход

### Стационарность

Перед тем, как перейти к моделированию, стоит сказать о таком важном свойстве временного ряда, как [**стационарность**](https://ru.wikipedia.org/wiki/Стационарность). 
Под стационарностью понимают свойство процесса не менять своих статистических характеристик с течением времени, а именно постоянство матожидания, постоянство дисперсии (она же [гомоскедастичность](https://ru.wikipedia.org/wiki/Гомоскедастичность)) и независимость ковариационной функции от времени (должна зависеть только от расстояния между наблюдениями). Наглядно можно посмотреть на эти свойства на картинках, взятых из поста [Sean Abu](http://www.seanabu.com/2016/03/22/time-series-seasonal-ARIMA-model-in-python/):

- Временной ряд справа не является стационарным, так как его матожидание со временем растёт

<img src="https://habrastorage.org/files/20c/9d8/a63/20c9d8a633ec436f91dccd4aedcc6940.png"/>

- Здесь не повезло с дисперсией - разброс значений ряда существенно варьируется в зависимости от периода

<img src="https://habrastorage.org/files/b88/eec/a67/b88eeca676d642449cab135273fd5a95.png"/>

- Наконец, на последнем графике видно, что значения ряда внезапно становятся ближе друг ко другу, образуя некоторый кластер, а в результате получаем непостоянство ковариаций

<img src="https://habrastorage.org/files/2f6/1ee/cb2/2f61eecb20714352840748b826e38680.png"/>

Почему стационарность так важна? По стационарному ряду просто строить прогноз, так как мы полагаем, что его будущие статистические характеристики не будут отличаться от наблюдаемых текущих. Большинство моделей временных рядов так или иначе моделируют и предсказывают эти характеристики (например, матожидание или дисперсию), поэтому в случае нестационарности исходного ряда предсказания окажутся неверными. К сожалению, большинство временных рядов, с которыми приходится сталкиваться за пределыми учебных материалов, стационарными не являются, но с этим можно (и нужно) бороться.

# 4) Machine Learning для временных рядов

# 5) To Infinity and Beyond